# Data preparation

Notebook with basic data preparation and train/test split for future experiments

In [1]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

## Data loading and preparation

Load raw data (has to be placed in "raw_data_path" directory)

In [2]:
raw_data_path = '~/Documents/STUDIA/Projekt_badawczy/moje/TEPS_Data_preparation_data_sample_info'

patients = pd.read_csv(os.path.join(raw_data_path, 'SampleInfo_short_multiclass_2022-10-14.tsv'), sep='\t')
patients = patients.reset_index(drop=True)

patients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1916 entries, 0 to 1915
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Sample.ID        1916 non-null   object 
 1   Group            1916 non-null   object 
 2   Stage            1648 non-null   object 
 3   Sex              1898 non-null   object 
 4   Age              1898 non-null   object 
 5   Lib.size         1916 non-null   int64  
 6   Description      107 non-null    object 
 7   Comments         0 non-null      float64
 8   IsNew            1916 non-null   object 
 9   PotentialIssues  36 non-null     object 
 10  TR               1916 non-null   object 
 11  RealLocation     1916 non-null   object 
 12  MultiGroup       1916 non-null   object 
 13  MultiGroup2      1916 non-null   object 
 14  MultiGroup3      1916 non-null   object 
 15  TrainTest        1916 non-null   object 
dtypes: float64(1), int64(1), object(14)
memory usage: 239.6+ KB


Check for different Stage and Group values

In [3]:
patients['Stage'].value_counts(dropna=False)

n.a.    794
IV      497
NaN     268
III     152
II      125
I        80
Name: Stage, dtype: int64

In [4]:
patients['Group'].value_counts(dropna=False)

NSCLC                             567
Asymptomatic controls             405
Pulmonary Hypertension            175
Ovarian cancer                    133
Glioma                            128
Pancreatic cancer                 123
Cholangiocarcinoma                 83
Multiple sclerosis                 83
Colorectal cancer                  80
Medically-intractable epilepsy     43
Endometrial cancer                 38
Angina pectoris                    26
Hepatocellular carcinoma           22
Esophageal carcinoma               10
Name: Group, dtype: int64

Filter out the "Asymptomatic control" values of Group and cases with undefined Stage

In [5]:
valid_stages = ('I', 'II', 'III', 'IV')
patients = patients.loc[patients['Stage'].isin(valid_stages)]

patients['Stage'].value_counts(dropna=False)

IV     497
III    152
II     125
I       80
Name: Stage, dtype: int64

In [6]:
patients['Group'].value_counts(dropna=False)

NSCLC                       404
Ovarian cancer              126
Pancreatic cancer           122
Cholangiocarcinoma           80
Colorectal cancer            63
Endometrial cancer           36
Hepatocellular carcinoma     14
Esophageal carcinoma          9
Name: Group, dtype: int64

Change data type of Age column to numeric

Change missing value markers to None

Rename column with patient IDs

Select only subset of available columns

In [7]:
patients['Age'] = pd.to_numeric(patients['Age'], errors='coerce')
patients.loc[patients['Sex'] == 'n.a.', 'Sex'] = None
patients = patients.rename(columns={'Sample.ID': 'ID'})

cols = [
    'ID',
    'Group',
    'Sex',
    'Age',
    'Stage',
    'RealLocation'
]
patients = patients.loc[:, cols]
patients.info()

print(patients)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 854 entries, 328 to 1755
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ID            854 non-null    object 
 1   Group         854 non-null    object 
 2   Sex           854 non-null    object 
 3   Age           853 non-null    float64
 4   Stage         854 non-null    object 
 5   RealLocation  854 non-null    object 
dtypes: float64(1), object(5)
memory usage: 46.7+ KB
                                ID               Group Sex   Age Stage  \
328                 MGH-NSCLC-L-74               NSCLC   M  36.0    IV   
329                 Vumc-NSCLC-092               NSCLC   F  81.0    IV   
331            MGH-NSCLC-L01-TR458               NSCLC   F  37.0    IV   
332            MGH-NSCLC-L11-TR477               NSCLC   M  54.0    IV   
334            MGH-NSCLC-L65-TR523               NSCLC   M  78.0    IV   
...                            ...                 .

Load and transpose a dataframe with marker values

In [8]:
markers = pd.read_csv(os.path.join(raw_data_path, 'Counts_prefiltered_multiclass_2022-10-14.tsv'), sep='\t')
markers = markers.T.reset_index(names='ID')
print(markers)

                     ID  ENSG00000000419  ENSG00000000938  ENSG00000001036  \
0     Vumc-HD-101-TR922         3.064289         3.834176         4.171537   
1     Vumc-HD-103-TR923         5.194380         6.964049         4.644469   
2     Vumc-HD-108-TR924         5.387337         7.608523         4.097419   
3     Vumc-HD-127-TR925         6.584300         5.626849         5.076153   
4     Vumc-HD-130-TR926         5.684044         5.990387         4.338011   
...                 ...              ...              ...              ...   
1911      TR3298-HD-NKI         5.015461         9.866231         6.145266   
1912      TR3481-HD-NKI         6.331082         9.694728         5.875447   
1913      TR3300-HD-NKI         5.133399        10.264108         6.606876   
1914      TR3296-HD-NKI         4.776702         9.413021         5.143608   
1915      TR4040-HD-NKI         6.642553         8.469676         4.599419   

      ENSG00000001461  ENSG00000001629  ENSG00000001631  ENSG00

In [9]:
col1_values = patients['ID'].tolist()
# usunięcie wierszy, w których wartość w kolumnie ID nie jest zawarta w kolumnie ID obiektu patients
markers = markers[markers['ID'].isin(col1_values)]
markers.info()
markers.head(5)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 854 entries, 328 to 1755
Columns: 4394 entries, ID to ENSG00000272168
dtypes: float64(4393), object(1)
memory usage: 28.6+ MB


,ID,ENSG00000000419,ENSG00000000938,ENSG00000001036,ENSG00000001461,ENSG00000001629,ENSG00000001631,ENSG00000002330,ENSG00000002549,ENSG00000002586,...,ENSG00000257267,ENSG00000257923,ENSG00000258890,ENSG00000263563,ENSG00000264538,ENSG00000266356,ENSG00000266714,ENSG00000269028,ENSG00000271043,ENSG00000272168
328,MGH-NSCLC-L-74,2.969287,4.409404,4.430591,5.180834,3.917734,4.250287,4.225813,4.274288,10.590616,...,6.208095,8.587142,2.377126,3.980061,4.122732,4.531599,3.441556,6.351508,5.381818,4.122732
329,Vumc-NSCLC-092,3.571121,5.485827,4.319007,3.512896,4.989149,4.163728,4.508574,4.971457,10.709285,...,6.217765,7.786546,4.130173,4.347874,4.130173,4.228193,4.196383,7.217239,5.925721,4.800937
331,MGH-NSCLC-L01-TR458,2.377126,3.401266,4.086679,2.724877,4.057926,3.219006,4.057926,4.220985,10.755117,...,6.393598,7.620465,2.377126,4.409539,3.067701,5.577047,3.760770,7.859787,6.340376,5.334665
332,MGH-NSCLC-L11-TR477,4.349115,4.926478,4.169191,4.330355,3.521482,4.125374,4.403788,6.093601,10.557800,...,6.761947,7.449481,4.056508,4.211490,3.481807,5.759331,3.351229,7.911409,6.342396,5.256122
334,MGH-NSCLC-L65-TR523,3.267676,3.424495,3.309719,5.342570,3.309719,3.875896,4.246368,4.373950,10.943650,...,6.444714,8.177893,3.618036,4.327609,3.267676,4.670803,2.662768,7.236297,5.856442,5.400439


In [10]:
markers.iloc[0:, 1:] = np.round(markers.iloc[0:, 1:], decimals=1)
markers.head(5)

,ID,ENSG00000000419,ENSG00000000938,ENSG00000001036,ENSG00000001461,ENSG00000001629,ENSG00000001631,ENSG00000002330,ENSG00000002549,ENSG00000002586,...,ENSG00000257267,ENSG00000257923,ENSG00000258890,ENSG00000263563,ENSG00000264538,ENSG00000266356,ENSG00000266714,ENSG00000269028,ENSG00000271043,ENSG00000272168
328,MGH-NSCLC-L-74,3.0,4.4,4.4,5.2,3.9,4.3,4.2,4.3,10.6,...,6.2,8.6,2.4,4.0,4.1,4.5,3.4,6.4,5.4,4.1
329,Vumc-NSCLC-092,3.6,5.5,4.3,3.5,5.0,4.2,4.5,5.0,10.7,...,6.2,7.8,4.1,4.3,4.1,4.2,4.2,7.2,5.9,4.8
331,MGH-NSCLC-L01-TR458,2.4,3.4,4.1,2.7,4.1,3.2,4.1,4.2,10.8,...,6.4,7.6,2.4,4.4,3.1,5.6,3.8,7.9,6.3,5.3
332,MGH-NSCLC-L11-TR477,4.3,4.9,4.2,4.3,3.5,4.1,4.4,6.1,10.6,...,6.8,7.4,4.1,4.2,3.5,5.8,3.4,7.9,6.3,5.3
334,MGH-NSCLC-L65-TR523,3.3,3.4,3.3,5.3,3.3,3.9,4.2,4.4,10.9,...,6.4,8.2,3.6,4.3,3.3,4.7,2.7,7.2,5.9,5.4


Sprawdzanie które geny mają najwięcej powtarzających się wartości

In [11]:
list_most_frequent_counts = []

# iteracja po kolumnach
for column in markers.columns:
    # zliczenie wystąpień każdej unikalnej wartości w kolumnie i wybór najczęściej występującej
    value_counts = markers[column].value_counts()
    most_frequent_value = value_counts.idxmax()
    most_frequent_count = value_counts.max()
    # wyświetlenie informacji o liczbie wystąpień najczęściej powtarzającej się wartości w kolumnie
    print(f"Najczęściej występująca wartość w kolumnie {column}: {most_frequent_value}")
    # dodanie słownika z nazwą kolumny i liczbą wystąpień najczęściej powtarzającej się wartości do listy
    list_most_frequent_counts.append({'Column': column, 'Most Frequent Value': most_frequent_value, 'Count': most_frequent_count})

# utworzenie obiektu DataFrame z listy słowników
most_frequent_counts = pd.DataFrame(list_most_frequent_counts)

# posortowanie DataFrame malejąco względem liczby wystąpień najczęściej powtarzającej się wartości
most_frequent_counts = most_frequent_counts.sort_values(by='Count', ascending=True)

most_freq = most_frequent_counts.head(200)

most_freq

Najczęściej występująca wartość w kolumnie ID: MGH-NSCLC-L-74
Najczęściej występująca wartość w kolumnie ENSG00000000419: 2.4
Najczęściej występująca wartość w kolumnie ENSG00000000938: 2.4
Najczęściej występująca wartość w kolumnie ENSG00000001036: 4.2
Najczęściej występująca wartość w kolumnie ENSG00000001461: 2.4
Najczęściej występująca wartość w kolumnie ENSG00000001629: 4.4
Najczęściej występująca wartość w kolumnie ENSG00000001631: 2.4
Najczęściej występująca wartość w kolumnie ENSG00000002330: 4.4
Najczęściej występująca wartość w kolumnie ENSG00000002549: 4.8
Najczęściej występująca wartość w kolumnie ENSG00000002586: 10.6
Najczęściej występująca wartość w kolumnie ENSG00000002822: 2.4
Najczęściej występująca wartość w kolumnie ENSG00000002834: 2.4
Najczęściej występująca wartość w kolumnie ENSG00000003056: 5.1
Najczęściej występująca wartość w kolumnie ENSG00000003402: 5.5
Najczęściej występująca wartość w kolumnie ENSG00000003436: 7.3
Najczęściej występująca wartość w kolumni

,Column,Most Frequent Value,Count
0,ID,MGH-NSCLC-L-74,1
4366,ENSG00000244734,8.7,22
484,ENSG00000076928,4.9,24
3205,ENSG00000163682,6.8,26
1246,ENSG00000109475,6.5,27
...,...,...,...
899,ENSG00000101596,5.0,40
3166,ENSG00000163131,2.4,40
265,ENSG00000057608,5.2,40
982,ENSG00000103512,5.4,40


Ograniczenie liczby genów do 200, w których wartości najczęściej się powtarzały

In [12]:
columns_to_keep = most_freq[most_freq['Column'] != 'ID']['Column'].tolist()
markers = markers[['ID'] + columns_to_keep]
markers.head(5)


,ID,ENSG00000244734,ENSG00000076928,ENSG00000163682,ENSG00000109475,ENSG00000142676,ENSG00000142937,ENSG00000081237,ENSG00000177600,ENSG00000105372,...,ENSG00000171490,ENSG00000204463,ENSG00000204628,ENSG00000082397,ENSG00000101161,ENSG00000101596,ENSG00000163131,ENSG00000057608,ENSG00000103512,ENSG00000156256
328,MGH-NSCLC-L-74,8.7,5.7,6.0,5.4,5.6,5.2,5.1,5.5,6.6,...,4.0,3.8,6.9,6.2,2.4,4.0,4.5,4.5,6.4,3.1
329,Vumc-NSCLC-092,6.9,6.6,7.4,7.2,7.4,6.7,7.5,7.3,8.4,...,4.7,4.8,8.0,6.4,4.8,5.6,5.8,5.5,5.5,4.5
331,MGH-NSCLC-L01-TR458,9.0,4.9,7.4,7.1,7.0,6.8,5.5,7.4,7.7,...,5.2,3.8,8.2,4.1,5.0,5.5,5.7,5.7,5.8,4.2
332,MGH-NSCLC-L11-TR477,10.8,6.0,7.9,7.9,7.7,7.3,6.5,7.4,8.9,...,6.5,3.8,8.7,6.0,5.6,4.8,6.2,7.0,8.2,4.3
334,MGH-NSCLC-L65-TR523,7.2,3.5,5.1,4.0,4.7,4.1,4.9,4.9,6.1,...,4.4,4.0,6.1,5.3,3.2,3.2,3.7,4.1,4.9,2.4


Merge patients with their markers values

In [13]:
# połączenie obiektów patients i markers po kolumnie ID
merged = patients.merge(markers, on='ID', how='left')
merged.head(5)

,ID,Group,Sex,Age,Stage,RealLocation,ENSG00000244734,ENSG00000076928,ENSG00000163682,ENSG00000109475,...,ENSG00000171490,ENSG00000204463,ENSG00000204628,ENSG00000082397,ENSG00000101161,ENSG00000101596,ENSG00000163131,ENSG00000057608,ENSG00000103512,ENSG00000156256
0,MGH-NSCLC-L-74,NSCLC,M,36.0,IV,MGH,8.7,5.7,6.0,5.4,...,4.0,3.8,6.9,6.2,2.4,4.0,4.5,4.5,6.4,3.1
1,Vumc-NSCLC-092,NSCLC,F,81.0,IV,VUMC,6.9,6.6,7.4,7.2,...,4.7,4.8,8.0,6.4,4.8,5.6,5.8,5.5,5.5,4.5
2,MGH-NSCLC-L01-TR458,NSCLC,F,37.0,IV,MGH,9.0,4.9,7.4,7.1,...,5.2,3.8,8.2,4.1,5.0,5.5,5.7,5.7,5.8,4.2
3,MGH-NSCLC-L11-TR477,NSCLC,M,54.0,IV,MGH,10.8,6.0,7.9,7.9,...,6.5,3.8,8.7,6.0,5.6,4.8,6.2,7.0,8.2,4.3
4,MGH-NSCLC-L65-TR523,NSCLC,M,78.0,IV,MGH,7.2,3.5,5.1,4.0,...,4.4,4.0,6.1,5.3,3.2,3.2,3.7,4.1,4.9,2.4


Save merged dataframe to file

In [14]:
cleaned_data_path = '~/Documents/STUDIA/Projekt_badawczy/moje/TEPS_Data_preparation_data_sample_info/cleaned'

merged.to_csv(os.path.join(cleaned_data_path, 'dataset.csv'), index=False, sep=';')

## Train test split

The dataset will be split to train/test sets in a stratified fashion based on two columns: Group and Stage.

Some types of cancer have only one sample with a specific stage, so it's impossible to split them in a stratified way - they will be randomly placed in a train or test set.

In [15]:
groups_and_stages = merged.apply(lambda row: f"{row['Group'].replace(' ', '_')}_{row['Stage']}", axis=1)
groups_and_stages_counts = groups_and_stages.value_counts()
groups_and_stages[groups_and_stages.isin(groups_and_stages_counts[groups_and_stages_counts == 1].index)] = 'temp'

train, test = train_test_split(merged, test_size=0.25, stratify=groups_and_stages)

Now both train and test sets are representative of (almost) every cancer/stage pairs available in the dataset.

In [16]:
train.apply(lambda row: f"{row['Group'].replace(' ', '_')}_{row['Stage']}", axis=1).value_counts()

NSCLC_IV                        247
Pancreatic_cancer_II             48
Colorectal_cancer_IV             40
NSCLC_III                        34
Cholangiocarcinoma_IV            33
Ovarian_cancer_III               32
Ovarian_cancer_IV                26
Pancreatic_cancer_III            23
Ovarian_cancer_I                 23
Pancreatic_cancer_IV             18
Endometrial_cancer_I             18
Cholangiocarcinoma_II            17
Ovarian_cancer_II                13
NSCLC_I                          11
NSCLC_II                         10
Hepatocellular_carcinoma_IV       8
Cholangiocarcinoma_III            7
Endometrial_cancer_III            6
Esophageal_carcinoma_III          6
Cholangiocarcinoma_I              4
Colorectal_cancer_III             4
Endometrial_cancer_II             3
Colorectal_cancer_II              2
Hepatocellular_carcinoma_III      2
Pancreatic_cancer_I               2
Hepatocellular_carcinoma_I        1
Colorectal_cancer_I               1
Esophageal_carcinoma_II     

In [17]:
test.apply(lambda row: f"{row['Group'].replace(' ', '_')}_{row['Stage']}", axis=1).value_counts()

NSCLC_IV                       83
Pancreatic_cancer_II           16
Colorectal_cancer_IV           14
NSCLC_III                      12
Ovarian_cancer_III             11
Cholangiocarcinoma_IV          11
Ovarian_cancer_IV               9
Ovarian_cancer_I                8
Pancreatic_cancer_III           8
Pancreatic_cancer_IV            6
Endometrial_cancer_I            6
Cholangiocarcinoma_II           5
NSCLC_I                         4
Ovarian_cancer_II               4
NSCLC_II                        3
Esophageal_carcinoma_III        2
Hepatocellular_carcinoma_IV     2
Cholangiocarcinoma_III          2
Endometrial_cancer_III          2
Colorectal_cancer_II            1
Pancreatic_cancer_I             1
Cholangiocarcinoma_I            1
Hepatocellular_carcinoma_II     1
Colorectal_cancer_III           1
Endometrial_cancer_II           1
dtype: int64

In [18]:
print(f'Train set size: {len(train)}')
print(f'Test set size: {len(test)}')

Train set size: 640
Test set size: 214


Save train/test sets to files

In [19]:
train.to_csv(os.path.join(cleaned_data_path, 'train.csv'), index=False, sep=';')
test.to_csv(os.path.join(cleaned_data_path, 'test.csv'), index=False, sep=';')